In [1]:
import pickle


with open('coord_list.pkl', 'rb') as f:
    data = pickle.load(f)


In [2]:
type(data[0])

astropy.coordinates.sky_coordinate.SkyCoord

In [3]:
from astroquery.simbad import Simbad
from astroquery.xmatch import XMatch
from astropy import units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

In [4]:
ra = []
dec = []
for coord in data:
    ra.append(coord.ra.value)
    dec.append(coord.dec.value)
tab = Table([ra, dec], names=('RA', 'Dec'))
tab

RA,Dec
float64,float64
88.69424375463352,-6.4776284709998615
81.57637442813676,-6.327234144349591
85.51903637148102,-6.408033712573103
90.1800004991527,-6.512782051634536
87.95891218259771,-6.46192491980695
85.25721850140215,-6.402046923892316
86.58262856929026,-6.431619801002863
86.6361400834329,-6.432196844615628
91.37104525614362,-6.538896770370073


In [12]:
tab.write('test.csv')

In [15]:
xmatch_tab = XMatch.query(cat1=open('test.csv'), 
                          cat2='SIMBAD', max_distance=0.5* u.arcsec,
                          colRA1='RA', colDec1='Dec')

In [16]:
xmatch_tab

angDist,RA,Dec,main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,nbref,ra_sexa,dec_sexa,main_type,other_types,radvel,redshift,sp_type,morph_type,plx,pmra,pmdec,size_maj,size_min,size_angle,B,V,R,J,H,K,u,g,r,i,z
float64,float64,float64,str21,float64,float64,float64,float64,int64,int64,str14,str14,str8,str65,float64,float64,str8,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.203009,84.43530168510243,-6.843529616368494,V* V589 Ori,84.4353419638,-6.8434898589,0.0,0.0,90,24,05 37 44.48207,-06 50 36.5635,Orion_V*,Or*|Em*|V* |*iN|IR |pr*|Y*O|Y*O|* |Y*O|X |Em*|Em*|Em*|* |* |*,23.047,7.7e-05,M1.0,--,2.4781,-0.23,0.1,--,--,--,16.91,16.124,--,12.283,11.474,11.207,18.459,17.088,15.554,14.736,14.32
0.193438,82.83908972737889,-7.39508036763199,WDS J05314-0724B,82.8390416667,-7.3950555556,--,--,172,1,05 31 21.37,-07 23 42.2,*in**,** |* |*i*,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
0.186806,82.96340914341795,-7.716174655284498,HD 36513,82.9634607631,-7.7161659339,0.0,0.0,90,16,05 31 51.23058,-07 42 58.1974,Star,* |IR,--,--,B9V,--,2.5365,0.53,-0.38,--,--,--,9.55,9.53,--,9.48,9.487,9.448,--,--,--,--,--
0.458512,90.73153127907604,-8.16726160830511,TYC 5349-595-1,90.7316437665,-8.1673234442,0.0,0.0,90,0,06 02 55.59450,-08 10 02.3644,Star,* |IR,-23.53,-7.8e-05,--,--,1.4208,-19.9,-34.57,--,--,--,11.76,11.17,--,10.16,9.877,9.804,--,--,--,--,--
0.358169,82.44963317006741,-8.958779487270053,TYC 5333-958-1,82.4497255005,-8.9588192409,0.001,0.001,90,0,05 29 47.93412,-08 57 31.7493,Star,* |IR,--,--,--,--,--,-3.7,-6.3,--,--,--,11.19,10.54,--,9.356,9.022,8.931,--,--,--,--,--
0.161367,82.81358487686472,-9.778893610238542,TYC 5337-151-1,82.8135789489,-9.7788491684,0.0,0.0,90,0,05 31 15.25895,-09 46 43.8570,Star,* |* |IR,14.31,4.8e-05,--,--,1.2632,0.23,-14.83,--,--,--,12.44,12.14,--,11.583,11.363,11.329,--,--,--,--,--
0.32137,90.56210040886707,-10.016488250896241,BD-10 1351,90.5620212909,-10.0165318252,0.0,0.0,90,29,06 02 14.88511,-10 00 59.5146,Ae*,V* |Em*|IR |* |pr*|Ir*|Ae*,--,--,B3V[e],--,1.1191,-1.75,1.79,--,--,--,10.54,10.4,--,9.193,8.334,7.464,--,--,--,--,--
0.23105,84.44324918195512,-9.962511744917638,CRTS J053746.3-095745,84.4432840201,-9.9625659829,0.0,0.0,90,1,05 37 46.38816,-09 57 45.2375,RRLyr,RR*,--,--,--,--,0.1203,5.43,-6.62,--,--,--,--,15.42,--,--,--,--,--,--,--,--,--
0.401938,91.1418590744326,-10.0503999071495,HD 41404,91.1419530529,-10.0504623783,0.0,0.0,90,6,06 04 34.06873,-10 03 01.6646,Star,* |IR,39.32,0.000131,K0III,--,2.973,-3.2,-4.74,--,--,--,9.73,8.58,--,6.687,6.16,6.023,--,--,--,--,--


In [18]:
orig_coord = SkyCoord(ra=tab['RA']*u.degree, dec=tab['Dec']*u.degree)
xmatch_coord = SkyCoord(ra=xmatch_tab['RA']*u.degree, dec=xmatch_tab['Dec']*u.degree)
idx, d2d, d3d = orig_coord.match_to_catalog_sky(xmatch_coord)

In [22]:
ind = d2d < 0.5 * u.arcsec

In [28]:
tab[ind]

RA,Dec
float64,float64
84.43530168510243,-6.843529616368494
82.83908972737889,-7.39508036763199
82.96340914341795,-7.716174655284498
90.73153127907604,-8.16726160830511
82.44963317006741,-8.958779487270053
82.81358487686472,-9.778893610238542
90.56210040886707,-10.016488250896241
84.44324918195512,-9.962511744917638
91.1418590744326,-10.0503999071495


In [5]:
orig_coord = SkyCoord(ra=tab['RA']*u.degree, dec=tab['Dec']*u.degree)

In [6]:
search_radius = [2 * u.arcsec] * len(orig_coord)

In [7]:
result_table = Simbad.query_region(orig_coord,
                                   radius=search_radius)

/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '5:54:46.6185 -6:28:39.4625': No astronomical object found :  
  (error.line, error.msg))
/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 6 raised an error (recorded in the `errors` attribute of the result table): '6:00:43.2001 -6:30:46.0154': No astronomical object found :  
  (error.line, error.msg))
/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 9 raised an error (recorded in the `errors` attribute of the result table): '5:46:19.8309 -6:25:53.8313': No astronomical object found :  
  (error.line, error.msg))
/usr/local/lib/python3.6/dist-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 12 raised an error (recorded in the `error

In [11]:
len(result_table)

3544

In [12]:
len(orig_coord)

3074

In [24]:
result_table.errors[0].msg.split(' ')[1][:-2]

'-6:28:39.4625'

In [16]:
?result_table